In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import sys
import os
import pickle
import pandas as pd
import json
import datetime
import time
import copy

py_file_location = './drive/My Drive/PrivatePackages'
sys.path.append(os.path.abspath(py_file_location))
from sklearn.model_selection import train_test_split

import JiaoCheng
import NingXiang

In [7]:
LABEL = 'trend'
MODEL = 'rfc'

In [8]:
curated_data = pd.read_csv('./drive/My Drive/LAB/Hackathon/data/curated_data.csv')
curated_data = curated_data.drop(columns=['closePrice', 'closePriceNorm', 'log_ret_normalised_by_day', 'log_ret', 'stock'])
curated_data['date'] = pd.to_datetime(curated_data['date'])

In [9]:
curated_data

,date,trend,closePrice_min_3,closePrice_max_3,closePrice_std_3,closePrice_mean_3,closePrice_slope_3,closePrice_skew_3,closePrice_abs_diff_mean_3,closePrice_diff_std_3,...,closePriceNorm_lag_10,log_ret_normalised_by_day_lag_10,mean_log_ret_lag_1,mean_closePrice_lag_1,mean_closePriceNorm_lag_1,mean_log_ret_normalised_by_day_lag_1,pos_log_ret_lag_1,pos_closePrice_lag_1,pos_closePriceNorm_lag_1,pos_log_ret_normalised_by_day_lag_1
0,2022-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-01,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,40.6170,0.241356,NaN,NaN,33.787500,0.529432,NaN
2,2023-01-02,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000525,40.6472,0.247020,-4.440892e-18,0.000569,33.838750,0.535137,0.007178
3,2023-01-03,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.000065,40.6172,0.231342,-8.881784e-18,-0.001615,33.735000,0.482227,-0.229975
4,2023-01-04,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.000623,40.5886,0.211599,1.776357e-17,-0.000616,33.710000,0.455742,0.001161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,2023-09-02,-1.0,43.11,43.66,0.248328,43.460000,0.011598,-0.685668,0.300,0.300,...,-0.136102,-2.032099,-0.002123,39.4416,-0.655234,5.773160e-17,-0.003932,35.979231,-0.815362,-0.320944
12496,2023-09-03,1.0,43.25,43.66,0.182635,43.506667,-0.009391,-0.667566,0.205,0.155,...,-0.539073,-1.515743,-0.001491,39.3646,-0.690649,4.662937e-17,-0.001368,35.899231,-0.860295,0.019382
12497,2023-09-04,1.0,43.02,43.61,0.242808,43.293333,-0.013529,0.262017,0.295,0.065,...,-0.377885,0.765811,-0.001420,39.3096,-0.716371,8.770762e-17,-0.004381,35.692308,-0.931225,-0.493615
12498,2023-09-05,-1.0,43.02,43.25,0.106145,43.100000,-0.005087,0.702403,0.120,0.120,...,-0.363231,0.030813,-0.000048,39.2904,-0.714126,-1.776357e-17,-0.001478,35.616923,-0.974622,-0.267420


In [10]:
date = [datetime.datetime(year=2023, month=1, day = 1) + datetime.timedelta(days=i) for i in range(250)]
val_date_start = date[175]
test_date_start = date[175+38]

In [ ]:
function_map = ['min', 'max', 'std', 'mean', 'slope', 'skew', 'abs_diff_mean', 'diff_std', 'sum']

for i in [3, 5, 10, 20]:
    print('Max Lag:', i)

    tmp_curated_data = copy.deepcopy(curated_data)

    for j in range(i+1, 11):
        tmp_curated_data = tmp_curated_data.drop(columns=[f'log_ret_lag_{j}', f'log_ret_normalised_by_day_lag_{j}', f'closePriceNorm_lag_{j}', f'closePrice_lag_{j}'], axis = 1)

    for j in [3, 5, 10, 20]:
        if j > i:

            columns = []
            for col in ['log_ret', 'closePrice', 'closePriceNorm', 'log_ret_normalised_by_day']:
                for func in function_map:
                    columns.append(f'{col}_{func}_{j}')
            tmp_curated_data = tmp_curated_data.drop(columns=columns, axis = 1)

    tmp_curated_data = tmp_curated_data.dropna()

    train = tmp_curated_data[(tmp_curated_data['date'] < val_date_start)]
    val = tmp_curated_data[(tmp_curated_data['date'] < test_date_start) & (tmp_curated_data['date'] >= val_date_start)]
    test = tmp_curated_data[(tmp_curated_data['date'] >= test_date_start)]

    jiaocheng = JiaoCheng.JiaoCheng()

    train_x = train.drop(columns=['date', 'trend'], axis = 1)
    train_y = train['trend']

    val_x = val.drop(columns=['date', 'trend'], axis = 1)
    val_y = val['trend']

    test_x = test.drop(columns=['date', 'trend'], axis = 1)
    test_y = test['trend']

    with open(f'./drive/My Drive/LAB/Hackathon/models/NingXiang_output_{LABEL}_lag_{i}.pickle', 'rb') as f:
        ningxiang_output = pickle.load(f)

    ningxiang_output = list(ningxiang_output.items())
    ningxiang_output = dict(ningxiang_output)

    jiaocheng = JiaoCheng.JiaoCheng()

    from sklearn.ensemble import RandomForestClassifier as RFC

    jiaocheng.read_in_model(RFC, 'Classification')

    jiaocheng.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

    del train_x
    del train_y
    del val_x
    del val_y

    parameter_choices = {
        'n_estimators': (25, 50, 100, 200, 400),
        'criterion': ('gini', 'entropy'),
        'max_features': (0.4, 0.55, 0.7, 0.85),
        'max_samples': (0.4, 0.55, 0.7, 0.85),
        'ccp_alpha': (0.0001, 0.001, 0.01, 0.1, 1, 10, 100)

    }

    jiaocheng.set_hyperparameters(parameter_choices)

    jiaocheng.set_non_tuneable_hyperparameters({'n_jobs': -1, 'random_state': 19260817})

    jiaocheng.set_features(ningxiang_output)

    jiaocheng.set_tuning_order(['features', 'criterion', 'max_features', 'max_samples', 'n_estimators', 'ccp_alpha'])

    jiaocheng.set_hyperparameter_default_values({
        'features': 0,
        'criterion': 'entropy',
        'max_features': 0.7,
        'max_samples': 0.7,
        'n_estimators': 50,
        'ccp_alpha': 0.01}
    )

    try:
        jiaocheng.read_in_tuning_result_df(f'./drive/My Drive/LAB/Hackathon/models/tuning/jiaocheng_{MODEL}_{LABEL}_lag_{i}.csv')
    except:
        pass

    jiaocheng.set_tuning_result_saving_address(f'./drive/My Drive/LAB/Hackathon/models/tuning/jiaocheng_{MODEL}_{LABEL}_lag_{i}.csv')
    jiaocheng.set_best_model_saving_address(f'./drive/My Drive/LAB/Hackathon/models/tmp_models/jiaocheng_{MODEL}_{LABEL}_lag_{i}.pickle')

    jiaocheng.tune()

    jiaocheng.view_best_combo_and_score()

    print('\n\n')

Max Lag: 3
JiaoCheng Initialised
JiaoCheng Initialised
Successfully read in model <class 'sklearn.ensemble._forest.RandomForestClassifier'>, which is a Classification model
Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data
Successfully recorded hyperparameter choices
Successfully recorded non_tuneable_hyperparameter choices
Successfully recorded tuneable feature combination choices and updated relevant internal structures
Successfully set tuning output address
Successfully set best model output address

Default combo: [1, 0, 2, 2, 2, 0] 


ROUND 1

Round 1 
Hyperparameter: features (index: 5) 

Trained and Tested combination 1 of 62720: (1, 0, 2, 2, 2, 0), taking 0.31 seconds to get val score of 0.5521
        Current best combo: (1, 0, 2, 2, 2, 0) with val score 0.5521
Trained and Tested combination 2 of 62720: (1, 0, 2, 2, 2, 1), taking 1.17 seconds to get val score of 0.5521
        Current best combo: (1, 0, 2, 2

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Trained and Tested combination 70 of 62720: (1, 0, 2, 2, 3, 0), taking 0.28 seconds to get val score of 0.5174
        Current best combo: (1, 0, 2, 2, 2, 0) with val score 0.5521


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Trained and Tested combination 71 of 62720: (1, 0, 2, 2, 4, 0), taking 0.28 seconds to get val score of 0.5174
        Current best combo: (1, 0, 2, 2, 2, 0) with val score 0.5521


...

Trained and Tested combination 207 of 215040: (0, 0, 2, 2, 4, 92), taking 11.9 seconds to get val score of 0.4826
        Current best combo: (0, 0, 2, 2, 2, 92) with val score 0.5595


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Trained and Tested combination 208 of 215040: (0, 0, 2, 2, 5, 92), taking 11.85 seconds to get val score of 0.4826
        Current best combo: (0, 0, 2, 2, 2, 92) with val score 0.5595


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Trained and Tested combination 209 of 215040: (0, 0, 2, 2, 6, 92), taking 13.6 seconds to get val score of 0.4826
        Current best combo: (0, 0, 2, 2, 2, 92) with val score 0.5595

Best combo after this hyperparameter: (0, 0, 2, 2, 2, 92) , NOT UPDATED SINCE LAST HYPERPARAMETER


ROUND 2

Round 2 
Hyperparameter: features (index: 5) 

Trained and Tested combination 210 of 215040: (0, 0, 2, 2, 2, 0), taking 0.14 seconds to get val score of 0.5521
        Current best combo: (0, 0, 2, 2, 2, 92) with val score 0.5595
Trained and Tested combination 211 of 215040: (0, 0, 2, 2, 2, 1), taking 0.45 seconds to get val score of 0.5521
        Current best combo: (0, 0, 2, 2, 2, 92) with val score 0.5595
Trained and Tested combination 212 of 215040: (0, 0, 2, 2, 2, 2), taking 0.58 seconds to get val score of 0.5447
        Current best combo: (0, 0, 2, 2, 2, 92) with val score 0.5595
Trained and Tested combination 213 of 215040: (0, 0, 2, 2, 2, 3), taking 0.59 seconds to get val score of 0.55